In [76]:
#Imports
import os
import numpy as np
import pandas as pd
import requests
import json
from datetime import datetime
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

load_dotenv()
%matplotlib inline

In [77]:
api = os.getenv("LUNARCRUSHKEY")

In [78]:
CoinsNews= "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,dot,xmr,xlm&sources=news,urls&limit=50000&type=chronological"

CoinsNewsRequest = requests.get(CoinsNews)
CoinsNewsData = CoinsNewsRequest.json()


In [79]:
#bitcoin (btc) | litecoin (ltc) | polkadot (dot) | monero (xmr) | stellar lumens (xlm)
#BTCNewsData["data"][1200]

In [90]:
sentiments = []

for article in CoinsNewsData["data"]:
    try:
        text = article["description"]
        date = article["time"]
        symbol = article["symbol"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        sentiments.append({
            "Date": date,
            "Ticker": symbol,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
            #"Text": text 
        })
        
    except AttributeError:
        pass 

# Create DataFrame
CoinsNewsDF = pd.DataFrame(sentiments)
CoinsNewsDF.tail()

,Date,Ticker,Compound,Positive,Negative,Neutral
2973,1491307200,XMR,-0.8768,0.034,0.294,0.672
2974,1429717212,XLM,0.2263,0.079,0.000,0.921
2975,1403908140,BTC,0.2023,0.113,0.077,0.811
2976,1387211177,BTC,0.4019,0.137,0.000,0.863
2977,1362657600,BTC,-0.5994,0.000,0.157,0.843


In [81]:
CoinsSocials= "https://api.lunarcrush.com/v2?data=feeds&key=cu8kjgxtqqixbwd9zi2lu9&symbol=btc,ltc,dot,xmr,xlm&limit=5000000&sources=twitter,reddit&type=chronological"

CoinsSMRequest = requests.get(CoinsSocials)
CoinsSMData = CoinsSMRequest.json()

In [82]:
#BTCSMData["data"][200]

In [89]:
sentiments = []

for article in CoinsSMData["data"]:
    try:
        text = article["body"]
        date = article["time"]
        symbol = article["symbol"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        sentiments.append({
            "Date": date,
            "Ticker": symbol,
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
            #"Text": text 
        })
        
    except AttributeError:
        pass 

# Create DataFrame
CoinsSMDF = pd.DataFrame(sentiments)
CoinsSMDF.tail()

,Date,Ticker,Compound,Positive,Negative,Neutral
4228,1607820453,LTC,0.0516,0.028,0.000,0.972
4229,1607820045,XMR,0.9739,0.170,0.064,0.766
4230,1607818095,BTC,0.0000,0.000,0.000,1.000
4231,1607818031,XLM,0.0000,0.000,0.000,1.000
4232,1607816103,LTC,0.8462,0.163,0.000,0.837


In [110]:
#Prep dataset

Coins_Sentiment = pd.concat([CoinsNewsDF,CoinsSMDF], axis="rows", ignore_index=True)
Coins_Sentiment['Date'] = pd.to_datetime(Coins_Sentiment['Date'],unit='s')

#Coins_Sentiment['Date'] = pd.to_datetime(Coins_Sentiment['Date']).dt.date

Coins_Sentiment = Coins_Sentiment.groupby(['Date','Ticker'],as_index=False).mean()
Coins_Sentiment = Coins_Sentiment.sort_values(by=['Date'], ascending=False,ignore_index=True)
Coins_Sentiment.set_index('Date', inplace=True, drop=True)

#BTC_Sentiment.reset_index(drop=True, inplace=True)
#BTC_Sentiment.reset_index(inplace=True)
#BTC_Sentiment.columns = ['Date', 'Ticker', 'Compound', 'Positive', 'Negative', 'Neutral']

Coins_Sentiment.head()

,Ticker,Compound,Positive,Negative,Neutral
Date,,,,,
2021-01-12 23:24:28,XLM,0.5859,0.173,0.0,0.827
2021-01-12 23:23:40,BTC,0.4215,0.412,0.0,0.588
2021-01-12 23:23:30,XLM,0.7506,0.363,0.0,0.637
2021-01-12 23:22:54,XLM,0.8020,0.507,0.0,0.493
2021-01-12 23:19:52,BTC,0.0000,0.000,0.0,1.000


In [111]:
##Create Sentiment Signal column

# create a list of our conditions
conditions = [
    (Coins_Sentiment['Compound'] < 0),
    (Coins_Sentiment['Compound'] == 0),
    (Coins_Sentiment['Compound'] > 0)
    ]

# create a list of the values we want to assign for each condition
values = ['-1', '0', '1']

# create a new column and use np.select to assign values to it using our lists as arguments
Coins_Sentiment['Sentiment Signal'] = np.select(conditions, values)

# display updated DataFrame
Coins_Sentiment.head()

,Ticker,Compound,Positive,Negative,Neutral,Sentiment Signal
Date,,,,,,
2021-01-12 23:24:28,XLM,0.5859,0.173,0.0,0.827,1
2021-01-12 23:23:40,BTC,0.4215,0.412,0.0,0.588,1
2021-01-12 23:23:30,XLM,0.7506,0.363,0.0,0.637,1
2021-01-12 23:22:54,XLM,0.8020,0.507,0.0,0.493,1
2021-01-12 23:19:52,BTC,0.0000,0.000,0.0,1.000,0


In [116]:
#Slice Bitcoin from DF
BitcoinDF = Coins_Sentiment.loc[Coins_Sentiment['Ticker'] == 'BTC']

#Resample data
BitcoinDF = BitcoinDF.resample('D').bfill()
BitcoinDF = BitcoinDF.sort_values(by=['Date'], ascending=False)

#BitcoinDF.tail()
#BitcoinDF.to_csv("../libs/data/BitcoinSentimentSignals.csv")

In [119]:
#Slice Litecoin from DF
LitecoinDF = Coins_Sentiment.loc[Coins_Sentiment['Ticker'] == 'LTC']

#Resample data
LitecoinDF = LitecoinDF.resample('D').bfill()
LitecoinDF = LitecoinDF.sort_values(by=['Date'], ascending=False)

#LitecoinDF.head()
#LitecoinDF.to_csv("../libs/data/LitecoinSentimentSignals.csv")

In [124]:
#Slice Polkadot from DF
PolkadotDF = Coins_Sentiment.loc[Coins_Sentiment['Ticker'] == 'DOT']

#Resample data
PolkadotDF = PolkadotDF.resample('D').bfill()
PolkadotDF = PolkadotDF.sort_values(by=['Date'], ascending=False)

#PolkadotDF.head(50)
#PolkadotDF.to_csv("../libs/data/PolkadotSentimentSignals.csv")

In [128]:
#Slice Monero from DF
MoneroDF = Coins_Sentiment.loc[Coins_Sentiment['Ticker'] == 'XMR']

#Resample data
MoneroDF = MoneroDF.resample('D').bfill()
MoneroDF = MoneroDF.sort_values(by=['Date'], ascending=False)

#MoneroDF.head()
#MoneroDF.to_csv("../libs/data/MoneroSentimentSignals.csv")

In [132]:
#Slice Stellar Lumens from DF
StellarLumensDF = Coins_Sentiment.loc[Coins_Sentiment['Ticker'] == 'XLM']

#Resample data
StellarLumensDF = StellarLumensDF.resample('D').bfill()
StellarLumensDF = StellarLumensDF.sort_values(by=['Date'], ascending=False)

#StellarLumensDF.head()
#StellarLumensDF.to_csv("../libs/data/StellarLumensSentimentSignals.csv")